2 - Named Entity Recognition:

This is just a raw example, how the entity recogniton works.
It uses the pre-trained language based models, based on news, wikipedia or website data. The initial creation of such model is expensive on computational resources. 
See this lik for more details: https://spacy.io/models
Note that this techique is language sensitive and you have to know in advance, what is the language of your dataset. Our training example is in English.

This is the link to Ukrainian model that you may use from the beginning and add additional precision by trianing it on the field data: https://spacy.io/models/uk

More reading: https://towardsdatascience.com/named-entity-recognition-ner-using-spacy-nlp-part-4-28da2ece57c6

First, set up the proper directory:
(You don't have to change the line below if you want to process the default dataset.)

In [57]:
#----
directory="./dataset-zelenskywarcriminal/all-lang/"
#----

In [58]:
import json
import gzip
import pandas as pd
from tqdm import tqdm
import os


df_ac=pd.DataFrame()
print("Consolidation of the files into one dataframe...")

for fil in tqdm(os.listdir(directory)):
    flen=len(os.listdir(directory))
    file=fil
    #print(file)
    filename=str(str(directory)+(os.fsdecode(fil)))
    if filename.endswith(".gz"):
        try:
            df_act = pd.read_json(filename,lines=True)
            df_ac=df_ac.append(df_act)
        except:
            continue
        
        continue
    else:
        continue

print("Dataframe generated!")
#print(df_ac.head())
#print(df_ac.columns)

100%|██████████| 3/3 [00:00<00:00, 74.57it/s]

Consolidation of the files into one dataframe...
Dataframe generated!


Check the counts of languages, present in the dataset:

For next processing, we chose to work with English only.
In the field you will need to use multiple language models, or good translator. In case of the latter, I recommend the Deepl AI.
https://www.deepl.com/en/docs-api Compared to Google Translate (as of 5-12-2022) Deepl seems to have better understanding for semantics and provides good API to work with.

In [59]:
#print(df_ac[["lang"]].head(5))
#print(df_ac["text"].head(2))
df_ac[["lang"]].value_counts().head()

lang
en      102
fr       45
it       16
es       12
qme      10
dtype: int64

Choose the language if interest, or keep the current preset for English:

In [60]:
LANG='en'

In [61]:
#Filter only the language with available model in spacy
df_ac=df_ac.loc[df_ac["lang"]==LANG]
df_ac[["lang"]].value_counts().head()

lang
en      102
dtype: int64

Codename for the English language model: "en_core_web_sm"
 (See this link to find more models: https://spacy.io/models)

In [62]:
LANG_MODEL_CODE='en_core_web_sm'

Run the code below to extract the Named entities, sentiment and keywords from the text:

The output is both CSV and "pickled" dataframe.  Since the tweets contains interpunction, that breaks the usual CSV formatting, the pickled output is also inlcuded. You can use this "pickled" file as an input for further analysis and the formatting will work.
Both output files will be saved to the same directory, that keeps this notebook.

In [63]:
def text_keywords_gen(user_input): 
    import requests
    import bs4
    from nltk.tokenize import sent_tokenize
    from gensim.summarization import summarize
    from gensim.summarization import keywords
    #text= input_texter(user_input)
    text=user_input
    #print("\nKeywords: ")
    #print(keywords(user_input).split("\n"))
    kwrds=keywords(user_input).split("\n")
    return kwrds

def sentiment_detector(span):
    import spacy
    from spacytextblob.spacytextblob import SpacyTextBlob
    nlp=spacy.load(LANG_MODEL_CODE)
    nlp.add_pipe('spacytextblob')
    doc=nlp(span)
    sentiment={"polarity":doc._.polarity,"subjectivity":doc._.subjectivity,"assessments":doc._.assessments}
    #print(str(sentiment))
    return sentiment

def ner_extract(sentence):
    #Named entity recognition in a span / sentence.
    import spacy
    nlp=spacy.load("en_core_web_lg")
    doc=nlp(sentence)
    entities=[]
    for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        entities.append([ent.text, ent.start_char, ent.end_char, ent.label_])
    return entities

def pickled_df_to_csv(infile_pickle,outfile_csv):
    import pandas as pd
    df=pd.read_pickle(infile_pickle)
    df.to_csv(outfile_csv)
    #print("CSV file saved to:" +" "+outfile_csv)

df_ac1=df_ac[['id','text',"lang"]]
print("Loaded slice of dataframe with text.")
df_ac["lang_model"]=df_ac["lang"].apply(spacy_lang_assign)
df_ac1['text_keywords']=df_ac1['text'].apply(text_keywords_gen)
print("Loaded keywords generated from the text column.")
df_ac1['sentiment']=df_ac1['text'].apply(sentiment_detector)
print('Sentiment detection finished.')
df_ac1['ner_detection']=df_ac1['text'].apply(ner_extract)
print("NER - Named Entity Recognition finished.")
print("Done!")
#print(df_ac1.head(3))

print("Saving df to pickle..")
infilef=directory+"/pickled-twitter-dump-df-keywords"
infilecsv=directory+"/df_ac-in-csv.csv"
df_ac.to_pickle(infilef,compression="infer",protocol=4)
print("Df pickled!")
print("Saving pickled DF to csv...")
pickled_df_to_csv(infilef,infilecsv)
print("df_ac saved to csv!")



Loaded slice of dataframe with text.
Loaded keywords generated from the text column.


<ipython-input-63-2d83fc9cbbb3>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ac1['text_keywords']=df_ac1['text'].apply(text_keywords_gen)
<ipython-input-63-2d83fc9cbbb3>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ac1['sentiment']=df_ac1['text'].apply(sentiment_detector)


Sentiment detection finished.
NER - Named Entity Recognition finished.
Done!
Saving df to pickle..
Df pickled!
Saving pickled DF to csv...
df_ac saved to csv!


<ipython-input-63-2d83fc9cbbb3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ac1['ner_detection']=df_ac1['text'].apply(ner_extract)


Note: In case of trouble, run also code below. It may occur on some systems, that you need to fix the environment in order to run such comptutational heavy operations like Named entity recognition.
In provided Virtual Machine the error should be fixed.

See the top 10 results for quick check:

(For more detalis on entity types, reffer to this link: https://spacy.io/usage/linguistic-features#named-entities )

In [64]:
df_ac1['ner_detection'].str[0].value_counts().head(20)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[american, 57, 65, NORP]                  3
[RT @TimBronson5:, 0, 16, ORG]            3
[RT @NoMoreNATO:, 0, 15, ORG]             3
[RT @Stop_This_Evil, 0, 18, ORG]          2
[Modi, 22, 26, PERSON]                    2
[Nazis, 60, 65, NORP]                     2
[@vonderleyen, 0, 12, ORG]                2
[ZelenskyWarCriminal, 19, 38, NORP]       2
[ZelenskyWarCriminal, 1, 20, ORG]         2
[RT, 0, 2, ORG]                           2
[RT @NTY57NTY, 0, 12, ORG]                2
[@GRDecter, 0, 9, PERSON]                 1
[Ukrainian, 22, 31, NORP]                 1
[Jewish, 53, 59, NORP]                    1
[RT @ianbremmer, 0, 14, ORG]              1
[@ZelenskyyUa @RishiSunak, 0, 24, ORG]    1
[RT @Rich_Gally_:, 0, 16, ORG]            1
[Nigel, 19, 24, PERSON]                   1
[millions, 100, 108, CARDINAL]            1
[@ZelenskyyUa Be, 14, 29, ORG]            1
Name: ner_detection, dtype: int64

Keywords results:

There are several ways of how to determine what is the keyword in particular text. This method is based on library called "gensim" and one of the advanatges is that it is language agnostic.


In [66]:
df_ac1['text_keywords'].str[0].value_counts().head(6)

                       29
zelenskywarcriminal     6
zelensky                4
times                   3
western                 2
eur                     2
Name: text_keywords, dtype: int64

Sentiment analysis:

The method uses combination of rules and pre-trained ML model to determine sentiment from so called "assessment words". Since the tweets do not always contain such words, sometimes it is not possible to detect the sentiment properly and the value remains empty.
Mode details here: https://spacy.io/universe/project/spacy-textblob


In [67]:
df_ac1['sentiment'].str[0].value_counts().head(20)

Series([], Name: sentiment, dtype: int64)